PREDIKSI CUACA DENGAN ARTIFICIAL NEURAL NETWORK

STEP BY STEP :
1. Upload dataset


In [1]:
import pandas as pd
import numpy as np
import random
import math

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# List target kolom yang ingin digunakan
target_columns = ['tanggal', 'tn', 'tx', 'tavg', 'rh_avg', 'rr', 'ss', 'ddd_x', 'ddd_car', 'ff_x', 'ff_avg']

# List target kolom nurmerik yang ingin digunakan
numeric_columns = ['tn', 'tx', 'tavg', 'rh_avg', 'r', 'ss', 'ddd_x', 'ff_x', 'ff_avg']

**Keterangan**:
- 8888: data tidak terukur
- 9999: Tidak Ada Data (tidak dilakukan pengukuran)
- Tn: Temperatur minimum	 (°C)
- Tx: Temperatur maksimum (°C)
- Tavg: Temperatur rata-rata (°C)
- RH_avg: Kelembapan rata-rata (%)
- RR: Curah hujan (mm)
- ss: Lamanya penyinaran matahari (jam)
- ddd_x: Arah angin saat kecepatan maksimum (°)
- ddd_car: Arah angin terbanyak (°)
- ff_x: Kecepatan angin maksimum (m/s)
- ff_avg: Kecepatan angin rata-rata (m/s)

In [4]:
# Ganti dengan nama file CSV yang ingin dibaca
df = pd.read_csv("/content/drive/MyDrive/Project Machine Learning Prediksi/Data Project ML/df_cuaca.csv")

# Mengganti nilai NaN dengan 0 di seluruh DataFrame
df = df.fillna(0)

df

,tanggal,tn,tx,tavg,rh_avg,rr,ss,ff_x,ddd_x,ff_avg,ddd_car,year,month
0,2023-01-01,24.5,31.0,27.5,79,25.2,0.0,8,300,4,NW,2023,1
1,2023-01-02,25.2,31.5,28.1,81,0.0,3.9,9,310,5,NW,2023,1
2,2023-01-03,25.5,32.8,28.4,77,0.0,2.2,5,300,4,NW,2023,1
3,2023-01-04,25.8,34.0,28.8,75,0.0,3.5,7,320,4,NW,2023,1
4,2023-01-05,25.8,34.2,29.1,72,0.0,7.6,7,300,3,NW,2023,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
876,2025-05-26,27.0,31.7,27.3,88,4.9,2.7,3,110,1,S,2025,5
877,2025-05-27,25.3,30.8,28.2,86,2.0,1.6,3,340,1,N,2025,5
878,2025-05-28,26.4,32.6,29.2,82,0.0,0.0,4,110,2,E,2025,5
879,2025-05-29,27.0,34.7,30.2,77,0.1,4.2,3,60,2,NE,2025,5


In [5]:
# Menambahkan kolom 'weathersit' berdasarkan nilai 'rr'
def categorize_weather(rr):
    if rr == 0:
        return 'cerah'
    elif 0.1 <= rr <= 5:
        return 'hujan ringan'
    elif 5.1 <= rr <= 10:
        return 'hujan sedang'
    elif rr > 10:
        return 'hujan lebat'
    else:
        return 'tidak diketahui' # Handle cases outside defined ranges

df['weathersit'] = df['rr'].apply(categorize_weather)

# Menambahkan kolom 'cd_ws' berdasarkan nilai 'weathersit'
def encode_weather(weathersit):
    if weathersit == 'cerah':
        return 1
    elif weathersit == 'hujan ringan':
        return 2
    elif weathersit == 'hujan sedang':
        return 3
    elif weathersit == 'hujan lebat':
        return 4
    else:
        return 0 # Handle cases outside defined categories

df['cd_ws'] = df['weathersit'].apply(encode_weather)

df

df

,tanggal,tn,tx,tavg,rh_avg,rr,ss,ff_x,ddd_x,ff_avg,ddd_car,year,month,weathersit,cd_ws
0,2023-01-01,24.5,31.0,27.5,79,25.2,0.0,8,300,4,NW,2023,1,hujan lebat,4
1,2023-01-02,25.2,31.5,28.1,81,0.0,3.9,9,310,5,NW,2023,1,cerah,1
2,2023-01-03,25.5,32.8,28.4,77,0.0,2.2,5,300,4,NW,2023,1,cerah,1
3,2023-01-04,25.8,34.0,28.8,75,0.0,3.5,7,320,4,NW,2023,1,cerah,1
4,2023-01-05,25.8,34.2,29.1,72,0.0,7.6,7,300,3,NW,2023,1,cerah,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
876,2025-05-26,27.0,31.7,27.3,88,4.9,2.7,3,110,1,S,2025,5,hujan ringan,2
877,2025-05-27,25.3,30.8,28.2,86,2.0,1.6,3,340,1,N,2025,5,hujan ringan,2
878,2025-05-28,26.4,32.6,29.2,82,0.0,0.0,4,110,2,E,2025,5,cerah,1
879,2025-05-29,27.0,34.7,30.2,77,0.1,4.2,3,60,2,NE,2025,5,hujan ringan,2


In [6]:
# Menambahkan kolom 'awan' berdasarkan nilai 'weathersit'
def determine_cloud(weathersit):
    if weathersit == 'cerah':
        return random.choice(['Cumulus', 'Cirrus'])
    elif weathersit == 'hujan ringan':
        return 'Stratus'
    elif weathersit == 'hujan sedang':
        return 'Nimbostratus'
    elif weathersit == 'hujan lebat':
        return 'Cumulonimbus'
    else:
        return 'Tidak ada awan' # Handle cases outside defined categories

df['awan'] = df['weathersit'].apply(determine_cloud)

# Menambahkan kolom 'cd_awan' berdasarkan nilai 'awan'
def encode_cloud(awan):
    if awan == 'Cumulus':
        return 1
    elif awan == 'Cirrus':
        return 2
    elif awan == 'Stratus':
        return 3
    elif awan == 'Nimbostratus':
        return 4
    elif awan == 'Cumulonimbus':
        return 5
    else:
        return 0 # Handle cases outside defined categories

df['cd_awan'] = df['awan'].apply(encode_cloud)

df

,tanggal,tn,tx,tavg,rh_avg,rr,ss,ff_x,ddd_x,ff_avg,ddd_car,year,month,weathersit,cd_ws,awan,cd_awan
0,2023-01-01,24.5,31.0,27.5,79,25.2,0.0,8,300,4,NW,2023,1,hujan lebat,4,Cumulonimbus,5
1,2023-01-02,25.2,31.5,28.1,81,0.0,3.9,9,310,5,NW,2023,1,cerah,1,Cumulus,1
2,2023-01-03,25.5,32.8,28.4,77,0.0,2.2,5,300,4,NW,2023,1,cerah,1,Cumulus,1
3,2023-01-04,25.8,34.0,28.8,75,0.0,3.5,7,320,4,NW,2023,1,cerah,1,Cumulus,1
4,2023-01-05,25.8,34.2,29.1,72,0.0,7.6,7,300,3,NW,2023,1,cerah,1,Cirrus,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
876,2025-05-26,27.0,31.7,27.3,88,4.9,2.7,3,110,1,S,2025,5,hujan ringan,2,Stratus,3
877,2025-05-27,25.3,30.8,28.2,86,2.0,1.6,3,340,1,N,2025,5,hujan ringan,2,Stratus,3
878,2025-05-28,26.4,32.6,29.2,82,0.0,0.0,4,110,2,E,2025,5,cerah,1,Cumulus,1
879,2025-05-29,27.0,34.7,30.2,77,0.1,4.2,3,60,2,NE,2025,5,hujan ringan,2,Stratus,3


In [7]:
# List target kolom yang ingin digunakan
target_columns = ['tavg', 'rh_avg', 'ff_avg',  'cd_awan', 'cd_ws']
df = df[target_columns]

# Renaming the columns
df = df.rename(columns={
    'tavg': 'temperatur',
    'rh_avg': 'kelembapan_udara',
    'ff_avg': 'kecepatan_angin',
    'cd_awan' : 'awan',
    'cd_ws' : 'cuaca'
})

features = ['temperatur', 'kelembapan_udara', 'kecepatan_angin', 'awan', 'cuaca']

dataset = df
data1 = dataset
data2 = data1
data3 = data2
data4 = data3

dataset.head()

,temperatur,kelembapan_udara,kecepatan_angin,awan,cuaca
0,27.5,79,4,5,4
1,28.1,81,5,1,1
2,28.4,77,4,1,1
3,28.8,75,4,1,1
4,29.1,72,3,2,1


3. Transformasi data ke array matrix


In [8]:
import numpy as np

temperature = np.array(dataset['temperatur'])
kelembapan = np.array(data1['kelembapan_udara'])
kecepatan_angin = np.array(data2['kecepatan_angin'])
awan = np.array(data4['awan'])
cuaca = np.array(data3['cuaca'])


matrix = [temperature,kelembapan,kecepatan_angin,awan,cuaca]
print(np.array(matrix,ndmin=2).T)

[[27.5 79.   4.   5.   4. ]
 [28.1 81.   5.   1.   1. ]
 [28.4 77.   4.   1.   1. ]
 ...
 [29.2 82.   2.   1.   1. ]
 [30.2 77.   2.   3.   2. ]
 [29.9 77.   1.   3.   2. ]]


4. inisialisasi komponen ANN
    a. Learning rate
    b. bias
    c. bobot

In [9]:
import random
import os

learning_rate = 1
bias = 1
weights = [random.random(),random.random(),random.random(),random.random(),random.random()]

5. buat fungsi Perceptron

In [10]:
def Perceptron(w, x, y, z, label):
    global weights
    global bias
    global learning_rate

    for i in range(len(label)):
        input1 = w[i]
        input2 = x[i]
        input3 = y[i]
        input4 = z[i]
        output = label[i]

        outputP = input1 * weights[0] + input2 * weights[1] + input3 * weights[2] + input4 * weights[3] + bias * weights[4]
        outputP = 1 / (1 + np.exp(-outputP))  # sigmoid

        error = output - outputP
        weights[0] += error * input1 * learning_rate
        weights[1] += error * input2 * learning_rate
        weights[2] += error * input3 * learning_rate
        weights[3] += error * input4 * learning_rate
        weights[4] += error * bias * learning_rate

6. buat perulangan untuk melatih data

In [11]:
for i in range(50):
    Perceptron(temperature, kelembapan, kecepatan_angin, awan, cuaca)

7. buat inputan

In [13]:
# Input dari pengguna
temp = float(input("Masukkan Nilai Temperature       : "))
kel = float(input("Masukkan Nilai Kelembapan        : "))
kec = float(input("Masukkan Nilai Kecepatan Angin   : "))
awan_kode = int(input("Masukkan Kode Awan              : "))

# Kategori suhu
if temp < 20:
    kategori_suhu = "Dingin"
elif temp < 30:
    kategori_suhu = "Sejuk"
else:
    kategori_suhu = "Panas"

# Menentukan jenis awan berdasarkan kode input
jenis_awan = "Tidak Diketahui" # Default value
if awan_kode == 1:
    jenis_awan = "Cumulus"
elif awan_kode == 2:
    jenis_awan = "Cirrus"
elif awan_kode == 3:
    jenis_awan = "Stratus"
elif awan_kode == 4:
    jenis_awan = "Nimbostratus"
elif awan_kode == 5:
    jenis_awan = "Cumulonimbus"

# Aturan prediksi cuaca
if kel < 30:
    prediksi = "Cerah"
    prob = 0.1
elif kel < 60:
    if kel > 30:
        prediksi = "Berawan"
        prob = 0.4
    else:
        prediksi = "Gerimis"
        prob = 0.6
else:
    if kel < 25:
        prediksi = "Gerimis"
        prob = 0.75
    else:
        prediksi = "Hujan Sedang"
        prob = 0.9

# Menyesuaikan prediksi berdasarkan Kode Awan
if awan_kode in [1, 2]: # Cumulus atau Cirrus (umumnya cuaca cerah/berawan ringan)
    if prediksi in ["Gerimis", "Hujan Sedang"]: # Jika prediksi awal adalah hujan/gerimis, mungkin turunkan prob
        prediksi = "Cerah Berawan" # Atau sesuaikan ke kategori yang lebih cerah
        prob *= 0.5 # Contoh: kurangi probabilitas hujan

elif awan_kode == 3: # Stratus (umumnya hujan ringan/gerimis)
    if prediksi == "Cerah": # Jika prediksi awal cerah, sesuaikan ke gerimis
        prediksi = "Gerimis"
        prob = max(prob, 0.5) # Contoh: pastikan prob minimal 0.5

elif awan_kode == 4: # Nimbostratus (umumnya hujan sedang)
     if prediksi in ["Cerah", "Berawan", "Gerimis"]: # Jika prediksi awal bukan hujan, sesuaikan ke hujan sedang
         prediksi = "Hujan sedang"
         prob = max(prob, 0.7) # Contoh: pastikan prob minimal 0.7

elif awan_kode == 5: # Cumulonimbus (umumnya hujan lebat/badai)
     if prediksi != "Hujan sedang": # Jika prediksi awal bukan hujan, sesuaikan ke hujan lebat
         prediksi = "Hujan lebat"
         prob = max(prob, 0.9) # Contoh: pastikan prob minimal 0.9

# Hitung kevalidan pakai Gaussian sederhana
if prediksi == "Cerah":
    tengah = 0.1
    batas_bawah, batas_atas = 0.0, 0.25
elif prediksi == "Berawan":
    tengah = 0.375
    batas_bawah, batas_atas = 0.26, 0.5
elif prediksi == "Gerimis":
    tengah = 0.63
    batas_bawah, batas_atas = 0.51, 0.75
else:
    tengah = 0.88
    batas_bawah, batas_atas = 0.76, 1.0

maks_jarak = (batas_atas - batas_bawah) / 2
# Menambahkan penanganan jika maks_jarak adalah 0 untuk menghindari pembagian dengan nol
if maks_jarak == 0:
    kevalidan = 0.0 # Atau nilai lain yang sesuai
else:
    kevalidan = math.exp(-((prob - tengah) ** 2) / (2 * (maks_jarak ** 2))) * 100

kevalidan = round(kevalidan, 2)

# Output
print("\n=== Hasil Prediksi Cuaca ===")
print(f"Temperature                 : {temp} °C ({kategori_suhu})")
print(f"Kelembapan                  : {kel} %")
print(f"Kecepatan Angin             : {kec} km/jam")
print(f"Awan                        : {awan_kode}")
print(f"Probabilitas Prediksi Cuaca : {prob:.2f}")
print(f"Prediksi Cuaca              : {prediksi}")
print(f"Tingkat Kevalidan Prediksi  : {kevalidan:.2f}%")

Masukkan Nilai Temperature       : 24
Masukkan Nilai Kelembapan        : 90
Masukkan Nilai Kecepatan Angin   : 25
Masukkan Kode Awan              : 4

=== Hasil Prediksi Cuaca ===
Temperature                 : 24.0 °C (Sejuk)
Kelembapan                  : 90.0 %
Kecepatan Angin             : 25.0 km/jam
Awan                        : 4
Probabilitas Prediksi Cuaca : 0.90
Prediksi Cuaca              : Hujan Sedang
Tingkat Kevalidan Prediksi  : 98.62%
